In [2]:
from bs4 import BeautifulSoup
import requests
import os 
import os.path

import time 
import json
import pandas as pd

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [4]:
header=['username','comment','label','date']
api="https://api-graphql.lefigaro.fr/"
id_comments="widget-comments_commentsQuery2_31d9f1fd61a3568936b76800aef3aade1b9002eee01930e2b9c499ceca28192e"
url=["https://www.lefigaro.fr/actualite-france/guerre-en-ukraine-un-non-ukrainien-sur-trois-parmi-les-refugies-en-france-20220308"]

In these URLs we can made search by selenium but we use manually with some filters depends on the country.

As you can say in these 2 URLs.

In [5]:
guerre_en_ukraine="https://recherche.lefigaro.fr/recherche/Guerre%20entre%20Ukraine%20et%20russie/?lieux_slug=ukraine&type=ART"
guerre_en_russie="https://recherche.lefigaro.fr/recherche/Guerre%20entre%20Ukraine%20et%20russie/?type=ART&lieux_slug=russie"

In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException,InvalidSessionIdException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC   
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service



In [7]:
def start():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    return driver

In [8]:
def get_articles(driver):
    # get the first article before click to load other articles
    webElement_articles=driver.find_elements(by=By.CSS_SELECTOR,value='#articles-list .fig-profil-mtpd')

    links_of_articles=[]

    for webElement_article in webElement_articles:
        link_article=webElement_article.find_element(by=By.CSS_SELECTOR,value='h2.fig-profil-headline a')
        
        links_of_articles.append(link_article.get_attribute('href'))

    return links_of_articles    
    
    # btn_more_article=driver.find_element(by=By.CSS_SELECTOR,value='button.js_startendeless')
    # btn_more_article.send_keys(Keys.ENTER)

In [9]:
# this function used for scroll all articles
def process_articles(link,driver):
    driver.get(link)
    time.sleep(2)  # Allow 2 seconds for the web page to open
    scroll_pause_time = 1 # You can set your own pause time. 
    screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web

    btn_more_article=driver.find_element(by=By.CSS_SELECTOR,value='button.js_startendeless')
    btn_more_article.send_keys(Keys.ENTER)

    i = 1

    while True:
        # scroll one screen height each time
        driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
        i += 1
        time.sleep(scroll_pause_time)
        # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        # Break the loop when the height we need to scroll to is larger than the total scroll height
        if (screen_height) * i > scroll_height:
            break
    links_of_articles=get_articles(driver)
    driver.close()
    return links_of_articles    

In [11]:
def get_data_content_id(driver):
    div_data_content=driver.find_element(by=By.CSS_SELECTOR, value='div.fig-comments')
    data_content_id=div_data_content.get_attribute('data-content-id')
    return data_content_id

In [13]:
def get_comments(data):
    i=1
    all_comments=[]
    comments=data['data']
    comments_data=comments['comments']
    if len(comments_data)==0:
        return all_comments
    for comment in comments_data:    
        text=comment['text']
        author=comment['author']
        date=comment['createdAt']
        if author:
            all_comments.append([author['username'],text,'',date])
        else:
            all_comments.append(['uknown',text,'',date])    
    
    return all_comments    

In [14]:

def send_request(data_content_id,page=1,id=id_comments):
    path="/graphql?id="+id_comments+"&variables={%22id%22:%22"+data_content_id+"%22,%22page%22:"+str(page)+"}"
    data_url=api+path
    response = requests.request("GET", data_url)
    return response

In [15]:

# This function we will use it to loop all the comments. page per page
def process_comments(data_content_id):
    comments_data=[]
    i=1
    while i!=0:
        response=send_request(page=i,data_content_id=data_content_id)
        if response.ok:
            raw_data=response.text

            data=json.loads(raw_data)
            # this instruction return list of comments
            comments=get_comments(data)

            if len(comments)==0:
                i=0
                continue
            else:
                for comment in comments:
                    comments_data.append(comment)
                i=i+1
                continue
        else:
            print('erreur in response')    
            i=0
            continue
    return comments_data              




## Get all articles from links (search by keywords)


In [105]:
links_russie_articles=process_articles(guerre_en_russie,start())



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/home/elkharroubi/.wdm/drivers/chromedriver/linux64/99.0.4844.51/chromedriver] found in cache


In [116]:
print(len(links_russie_articles))
print(links_russie_articles[0])

862
http://www.lefigaro.fr/international/la-cyberguerre-fait-rage-entre-la-russie-et-l-ukraine-20220301


In [128]:
pd.DataFrame(links_russie_articles,columns=['Links']).to_csv('./data/links_russie_articles.csv',index=False)

In [109]:
links_ukraine_articles=process_articles(guerre_en_ukraine,start())



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [/home/elkharroubi/.wdm/drivers/chromedriver/linux64/99.0.4844.51/chromedriver] found in cache


In [117]:
print(len(links_ukraine_articles))
print(links_ukraine_articles[0])

811
http://www.lefigaro.fr/international/la-cyberguerre-fait-rage-entre-la-russie-et-l-ukraine-20220301


In [129]:
pd.DataFrame(links_ukraine_articles,columns=['Links']).to_csv('./data/links_ukraine_articles.csv',index=False)

## Processus links  from file

In [16]:
def process_link(link,driver):
    # first we should check if there is comments on the article
    comments=[]
    driver.get(link)
    #time.sleep(1)
    number_comment=int(driver.find_element(by=By.CSS_SELECTOR,value='p.fig-comments-tabbar__small').text)
    print(number_comment)
    if number_comment==0 or number_comment==1:
        return comments
    # second we should get the id of every page
    # this id we give us the acces to comments
    data_content_id=get_data_content_id(driver)
    comments=process_comments(data_content_id)
    return comments


In [17]:
links_ukraine_articles=pd.read_csv('./data/links_ukraine_articles.csv')

In [18]:
links_ukraine_articles.head()

,Links
0,http://www.lefigaro.fr/international/la-cyberg...
1,http://www.lefigaro.fr/international/guerre-en...
2,http://www.lefigaro.fr/international/guerre-en...
3,http://www.lefigaro.fr/international/explicati...
4,http://www.lefigaro.fr/secteur/high-tech/guerr...


In [ ]:
driver=start()
data_comments=[]
links_exception=[]
i=0
for link in links_ukraine_articles.values:
    try:
        i=i+1
        comments=process_link(link[0],driver)
        if len(comments)==0:
            continue
        for comment in comments:
            data_comments.append(comment)
    except WebDriverException:
        links_exception.append(link[0])
        #driver.close()
    except InvalidSessionIdException:
        links_exception.append(link[0])
        driver.close()
        continue



In [ ]:
pd.DataFrame(links_ukraine_articles,columns=header).to_csv('./data/comments_ukraine_articles.csv',index=False)

In [228]:
pd.DataFrame(data_comments,columns=header).to_csv('./data/comments_ukraine_articles.csv',index=False)